<a href="https://colab.research.google.com/github/puchana123/Waste-Separation-using-YOLO11/blob/main/%E0%B8%B7notebooks/TrainModel_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TrainModel: The Waste Speration using YOLO11

Reference from [Roboflow Train YOLO11 Notebook](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolo11-object-detection-on-custom-dataset.ipynb) and [Youtube:Roboflow YOLOv11 guide](https://www.youtube.com/watch?v=etjkjZoG2F0)

## Setup

### Configure API keys

To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

### Install YOLO11 via Ultralytics

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
# prevent ultralytics from tracking your activity
!yolo settings sync=False
import ultralytics
ultralytics.checks()

## Get the custom dataset from Roboflow

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. When downloading, make sure to select the `yolov11` export format.

the path should look like this `datasets/file_name/test/images`

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata # use Secrets in your google Colab
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY') # Use your Roboflow api key
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("datamodel-fhfwy").project("wastedetection16-8w08u") # The project from your Roboflow
version = project.version(5)
dataset = version.download("yolov11")

In [ ]:
dataset.location

## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11n.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 batch=16 patience=50 project=waste_model name=waste_v2 plots=True

**NOTE:** The results of the completed training are saved in `{HOME}/runs/detect/train/`. Let's examine them.

In [ ]:
!ls {HOME}/waste_model/waste_v2

In [ ]:
# Display Confusion Matrix
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/waste_model/waste_v2/confusion_matrix.png', width=600)

## Validate fine-tuned model

In [ ]:
!yolo task=detect mode=val model=/content/weights/best.pt data=/content/datasets/WasteDetection16-4/data.yaml

In [ ]:
# Display Confusion Matrix
from IPython.display import Image as IPyImage

IPyImage(filename=f'/content/datasets/runs/detect/val/confusion_matrix.png', width=600)

In [ ]:
# Display Predicted Images
from IPython.display import Image as IPyImage

IPyImage(filename=f'/content/datasets/runs/detect/val/val_batch1_pred.jpg', width=600)

## Predict the images

In [ ]:
!yolo task=detect mode=predict model=/content/waste_model/waste_v2/weights/best.pt source=/content/test

In [ ]:
# Display Predicted Image
from IPython.display import Image as IPyImage

IPyImage(filename=f'/content/runs/detect/predict/image_name.jpg', width=600)

In [ ]:
# Display Predicted Images in Grid
import glob
from IPython.display import Image, display

image_files = glob.glob('/content/runs/detect/predict/*.jpg')
for image_file in image_files:
    display(Image(filename=image_file, width=600))